<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____
<a id='home'></a>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/introOptimization/blob/main/Intro_To_Optimization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Introduction to Optimization for Decision Making


In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 1: Maximization/Minimization 

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.
Then, call the library:

In [ ]:
# pip show glpk pulp
# pip install glpk pulp

In [ ]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [ ]:
model = pp.LpProblem(name='refinery-problem', # just the name
                     sense=pp.LpMaximize) # type of problem

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [ ]:
# how much gas?
Gas = pp.LpVariable(name="Gas",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # here: you accept decimal values

# how much oil?
Oil = pp.LpVariable(name="Oil",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients__: 

In [ ]:
GasCoeff=1.9
OilCoeff=1.5
obj_func = GasCoeff*Gas + OilCoeff*Oil

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [ ]:
# SUBJECT TO:
C1= pp.LpConstraint(name='Gas Constraint',   # just the name
                    e= 1*Gas - 2*Oil, rhs=0, # linear combination of constraint and rhs 
                    sense=pp.LpConstraintGE) # 'rule' >= 0 (LpConstraintGE)
C2= pp.LpConstraint(name='Oil Constraint',
                    e= 1*Oil, rhs=3000000,
                    sense=pp.LpConstraintGE) # 'rule' >= 3000000 (LpConstraintGE)
C3= pp.LpConstraint(name='Demand Constraint',
                    e= 1*Gas, rhs=6400000,
                    sense=pp.LpConstraintLE, )# 'rule' <= 6400000 (LpConstraintLE)

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [ ]:
model += obj_func
model += C1
model += C2
model += C3


6. **Solve the MODEL**: Notice we are not using the _default solver_, we are explicitly usig **COIN_CMD**:

In [ ]:
solver_list = pp.listSolvers()
print(solver_list)

In [ ]:
solverToUse = pp.COIN_CMD(msg=False)
model.solve(solver=solverToUse);

You can create a summary like this:

In [ ]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

In [ ]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

<div class="alert-success">

<strong>Exercise: The diet problem</strong> 

In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 2: Multicriteria Decision-Making

In [ ]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7GL_wF1eKRO0JgEUyIx5cxXUhTQ8ZM4F3AE1MLr7GYG33dwEobrLo6O2MaV2d7Cv47TaTgHghkhrV/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

1. Prepare data file with the comparissons:

In [ ]:
%%html

<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSeUfh-DtfAAvEecirNS7Qs2qN4npmNfRiw9JvKmRpq88snVc8HJBlru2cyPy8lsQflSxlnx6U-IePw/pubhtml?widget=true&amp;headers=false" width="600" height="300" ></iframe>

2. Get the data (Excel)

In [ ]:
# the link to the data

linkGoogle='https://docs.google.com/spreadsheets/d/e/2PACX-1vSeUfh-DtfAAvEecirNS7Qs2qN4npmNfRiw9JvKmRpq88snVc8HJBlru2cyPy8lsQflSxlnx6U-IePw/pub?output=xlsx'# the link to the data

3. Open each sheet:

In [ ]:
# opening the comparissons

import pandas as pd

pairwise_age=pd.read_excel(linkGoogle,sheet_name='age', index_col=0)
pairwise_experience=pd.read_excel(linkGoogle,sheet_name='experience', index_col=0)
pairwise_education=pd.read_excel(linkGoogle,sheet_name='education', index_col=0)
pairwise_charisma=pd.read_excel(linkGoogle,sheet_name='charisma', index_col=0)
pairwise_criteria=pd.read_excel(linkGoogle,sheet_name='criteria', index_col=0)

You may want to check the structure:

In [ ]:
pairwise_criteria

4. Transform all matrices into pairwise comparissons:

In [ ]:
import networkx as nx

G_age = nx.from_pandas_adjacency(pairwise_age,create_using=nx.MultiDiGraph())

# pairwise
G_age.edges(data=True)

In [ ]:
# comparissons for age as dict
age_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_age.edges(data=True) if e[0]!= e[1]}
age_comparisons

In [ ]:
# the remaining comparissons:

G_exp = nx.from_pandas_adjacency(pairwise_experience,create_using=nx.MultiDiGraph())
experience_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_exp.edges(data=True) if e[0]!= e[1]}

G_edu = nx.from_pandas_adjacency(pairwise_education,create_using=nx.MultiDiGraph())
education_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_edu.edges(data=True) if e[0]!= e[1]}

G_cha = nx.from_pandas_adjacency(pairwise_charisma,create_using=nx.MultiDiGraph())
charisma_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_cha.edges(data=True) if e[0]!= e[1]}

In [ ]:
# take a look
[age_comparisons, experience_comparisons,education_comparisons,charisma_comparisons]

In [ ]:
# now the criteria

G_CRIT = nx.from_pandas_adjacency(pairwise_criteria,create_using=nx.MultiDiGraph())
criteria_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_CRIT.edges(data=True) if e[0]!= e[1]}
criteria_comparisons

5. Apply the Algorithm

In [ ]:
## install
# !pip install ahpy

In [ ]:
# input each comparisson

import ahpy

experience = ahpy.Compare('experience', experience_comparisons, precision=3, random_index='saaty')
education = ahpy.Compare('education', education_comparisons, precision=3, random_index='saaty')
charisma = ahpy.Compare('charisma', charisma_comparisons, precision=3, random_index='saaty')
age = ahpy.Compare('age', age_comparisons, precision=3, random_index='saaty')
criteria = ahpy.Compare('criteria', criteria_comparisons, precision=3, random_index='saaty')

6. Create hierarchy:

In [ ]:
criteria.add_children([experience, education, charisma, age])

7. See result:

In [ ]:
print(criteria.target_weights)

8. Assess consistency

In [ ]:
## We should review comparissons if greater than 0.1!
[(val.name,val.consistency_ratio) for val in [experience, education, charisma, age, criteria]]

<div class="alert-success">

<strong>Exercise: Choosing a country for a Master Program</strong> 

- Make a group of 4 people from this course.
- If you have the criteria: cost of living, language difficulty, possibilities to get a job in that country after studies are finished.
- If you have the alternatives: Brazil, Spain, USA, Germany.
- Create an AHP model and get the ranking.

You can follow this [example](https://en.wikipedia.org/wiki/Analytic_hierarchy_process_%E2%80%93_leader_example).
If you have a better idea, you can use it instead.

# Part 3: Benchmarking

Imagine you have this [information](https://www.sciencedirect.com/science/article/abs/pii/S0377221711007168):

In [65]:
airline=pd.read_csv("airlines_data.csv")
airline

,name,Aircraft,Fuel,Employee,Passenger,Freight
0,A,109,392,8259,23756,870
1,B,115,381,9628,24183,1359
2,C,767,2673,70923,163483,12449
3,D,90,282,9683,10370,509
4,E,461,1608,40630,99047,3726
5,F,628,2074,47420,128635,9214
6,G,81,75,7115,11962,536
7,I,153,458,10177,32436,1462
8,J,455,1722,29124,83862,6337
9,K,103,400,8987,14618,785


The first three variables (Aircraft,Fuel,Employee) represent **inputs** and the last two ones represent **outputs**. If that is so, there should be a way to compute some measure of efficiency: the ratio **output/input**.

Let's compute some ratios:

In [ ]:
# ratio passenger employee:
airline['rate_ClientsByEmployee']=(airline.Passenger/airline.Employee)
airline['rate_CargoByFleet']=(airline.Freight/airline.Aircraft)

Let me plot those ratios:

In [ ]:
import altair as alt

points = alt.Chart(airline).mark_point().encode(
    x='rate_ClientsByEmployee:Q',
    y='rate_CargoByFleet:Q'
)

text = points.mark_text(
    align='right',
    baseline='middle',
    dx=-7
).encode(
    text='name'
).interactive()

points + text

Which one is more efficient? As you see, one airline might not be good in both ratios:

In [ ]:
airline[['name','rate_ClientsByEmployee','rate_CargoByFleet']].sort_values(by='rate_ClientsByEmployee',ascending=False).head()

In [ ]:
airline[['name','rate_ClientsByEmployee','rate_CargoByFleet']].sort_values(by='rate_CargoByFleet',ascending=False).head()

Let me show you the **envelope**:

In [ ]:
Best_ClientsByEmployee=airline.rate_ClientsByEmployee.idxmax()
Best_CargoByFleet=airline.rate_CargoByFleet.idxmax()

frontier1=airline.loc[Best_ClientsByEmployee,['rate_ClientsByEmployee','rate_CargoByFleet']].to_list()
frontier2=airline.loc[Best_CargoByFleet,['rate_ClientsByEmployee','rate_CargoByFleet']].to_list()

#parallels
frontier1v=[frontier1[0],0]
frontier2h=[0,frontier2[1]]

#then
envelope=pd.DataFrame([frontier2h,frontier2,frontier1,frontier1v],columns=['x','y'])
envelope

Updating the plot:

In [ ]:
points + text + alt.Chart(envelope).mark_line(color='red').encode(
    x='x',
    y='y',
)

The presence of several units (DMUs), several inputs, and several outputs makes it difficult to judge who is doing better. This an optimization problem that may be carried out using **Pyfrontier**:

In [ ]:
## installation
# pip install Pyfrontier

In [ ]:
airlineInput=airline.columns[1:4].to_list()
airlineOutput=airline.columns[4:6].to_list()

In [ ]:
from Pyfrontier.frontier_model import EnvelopDEA

dea_air_vrs_in = EnvelopDEA("VRS", "in")
dea_air_vrs_in.fit(
    inputs=airline[airlineInput].to_numpy(),
    outputs=airline[airlineOutput].to_numpy()
)

In [ ]:
VRS_in=pd.DataFrame()
vrs_in_names_output=['_'.join((a,'vrs_in_sl')) for a in airlineOutput]
vrs_in_names_input=['_'.join((a,'vrs_in_sl')) for a in airlineInput]

VRS_in['vrs_in']=[r.score for r in dea_air_vrs_in.result]
VRS_in[vrs_in_names_input]=[[round(v,2) for v in r.x_slack] for r in dea_air_vrs_in.result]
VRS_in[vrs_in_names_output]=[[round(v,2) for v in r.y_slack] for r in dea_air_vrs_in.result]
VRS_in['efficient_vrs_in']=[r.is_efficient for r in dea_air_vrs_in.result]
VRS_in.set_index(airline.name,inplace=True)
VRS_in

In [ ]:
VRS_in.vrs_in

In [ ]:
airline['vrsin']=[r.score for r in dea_air_vrs_in.result]
airline

In [ ]:
# !pip install py4etrics


In [ ]:
import numpy as np # linear algebra
from py4etrics import tobit
import statsmodels.api as sm

In [ ]:
np.where(airline['vrsin']==1, 1, 0)

In [ ]:
airline.loc[:,'Aircraft':'Freight']

In [ ]:
airline['censored'] =np.where(airline['vrsin']==1, 1, 0)
cens = airline['censored']#.values
endog = airline.loc[:,'vrsin']
exog = airline.loc[:,'Aircraft':'Freight']

tobit_res = tobit.Tobit(endog, exog, cens,right=1).fit()
tobit_res.summary()

In [ ]:
airline.to_csv('airline_all.csv')

In [ ]:
import pandas as pd
df = pd.read_csv("georgian_bay_secchi.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.secchi[df.secchi.str.contains('>')]

In [ ]:
import numpy as np
df['censored'] = np.where(df['secchi'].astype('str').str.contains(">"), 1, 0)
df['secchi_depth'] = df['secchi'].astype(str).str.replace(r"\>", "", regex=True).astype("float")
df['intercept'] = 1

In [ ]:
cens = df['censored'].values
endog = df['secchi_depth']
exog = df[['intercept','alkalinity', 'chloride', 'carbon_organic', 'nitrogen', 'sulphate']]

In [ ]:
cens

In [ ]:
endog

In [ ]:
exog

In [ ]:
from py4etrics import tobit
import statsmodels.api as sm
tobit_res = tobit.Tobit(endog, exog, cens).fit()
tobit_res.summary()

In [ ]:
import pandas as pd

ranch=pd.read_excel("ranchChain.xlsx",'data')

In [ ]:
ranch

In [ ]:
## installation
# !pip install Pyfrontier

In [ ]:
from Pyfrontier.frontier_model import EnvelopDEA

dea_ranch_vrs_in = EnvelopDEA("VRS", "in")
dea_ranch_vrs_in_fit=dea_ranch_vrs_in .fit(
    inputs=ranch[['HoursOps',	'FTE',	'Supplies']].to_numpy(),
    outputs=ranch[['WeeklyProfit','MarketShare','GrowthRate']].to_numpy()
)

In [ ]:
dea_ranch_vrs_in.result

In [ ]:
ranch['vrs_in']=[r.score for r in dea_ranch_vrs_in.result]
ranch[['HoursOps_vrs_in_sl','FTE_vrs_in_sl','Supplies_vrs_in_sl']]=[[round(v,2) for v in r.x_slack] for r in dea_ranch_vrs_in.result]
ranch[['WeeklyProfit_vrs_in_sl','MarketShare_vrs_in_sl','GrowthRate_vrs_in_sl']]=[[round(v,2) for v in r.y_slack] for r in dea_ranch_vrs_in.result]
ranch['WeeklyProfit']=[r.is_efficient for r in dea_ranch_vrs_in.result]
ranch.set_index('branch').loc[:,'vrs_in':]

In [ ]:
import statsmodels.api as sm

In [ ]:
!pip install statsmodels